# TFM. Modelo predictivo llamadas Call Center

### Escenario

**Angel24**, empresa mallorquina con mas de 40 años en el sector, dispone de un servicio de Call Center, donde se realizan campañas telefonicas de todo tipo (*llamadas entrantes y salientes*) 

En base a la experiencia, gestionar el equipo de personas necesario para prestar un servicio adecuado en  base a unos crioteroios de calidad es dificil, sobretodo en las llamadas entrantes ya que no siempre se comportan igual.

Una de las campañas mas importantes que tiene este departamento de Call Center, es prestar atencion telefonica a los clientes de diferentes franquicias de **MRW**. Es un servicio de llamadas entrantes (*Inbound*)

Mediante el estudio de los datos historicos, se pretende dimensionar al equipo por slots para poder prestar el mejor servicio posible

### Fuentes de los datos

Los datos de los que disponemos son:
* Historial de llamadas
* Historial de envios que llegan a las oficinas


De ambos ficheros, disponemos informacion de 1 año.

### Pregunta objetivo

En base a lo descrito hasta el momento, queremos saber:

***¿Cuantas llamadas tendremos mañana de 10 a 12?*** Generalizando, saber el numero de llamadas por dia y hora en funcion de los envios que nos lleguen a las islas.

## Importacion de librerias 

In [1]:
from os import getenv
import pymssql
import pandas as pd

pd.options.display.max_columns = None

### Importacion de los datos de llamadas

In [2]:
df_calls = pd.read_csv('data/calls.csv', low_memory=False)

In [3]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186208 entries, 0 to 186207
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          186208 non-null  int64  
 1   idTransaccion       186208 non-null  float64
 2   tInicio             97149 non-null   object 
 3   tFinal              186208 non-null  object 
 4   estado              186208 non-null  float64
 5   tProximoContacto    3 non-null       object 
 6   tReplanificacion    3701 non-null    object 
 7   nTAdmin             186208 non-null  int64  
 8   nTAdminExcedido     186208 non-null  int64  
 9   nTQ                 186208 non-null  int64  
 10  idSujeto            186208 non-null  float64
 11  sIdOriginal         186208 non-null  int64  
 12  sNombreCliente      186208 non-null  object 
 13  sApellido1Cliente   186208 non-null  object 
 14  sApellido2Cliente   0 non-null       float64
 15  sTelefonoCliente    0 non-null    

El DataFrame cargado de las llamadas tiene 28 columnas:
* idTransaccion = Identificador de la transaccion
* tInicio = Fecha hora inicio de la transaccion con agente
* tFinal = Fecha hora fin de la transaccion con agente
* estado = ...no sabemos siempre 1
* tProximoContacto = Fecha hora reprogramaciones
* tReplanificacion = Fecha hora reprogramaciones
* nTAdmin = Tiempo administrativo
* nTAdminExcedido = Tiempo administrativo exedido
* nTQ = Tiempo en cola
* idSujeto = Identificador del llamante
* sIdOriginal = Identificador del llamante
* sNombreCliente = Identificador del llamante
* sApellido1Cliente = Identificador del llamante
* sApellido2Cliente = Identificador del llamante
* sTelefonoCliente = Identificador del llamante
* idServicio = Identificador de servicio (grupo campañas)
* idCampanya = Identificador campañas
* idAgente = Identificador ragentes
* idSegmento = Segmentacion de llamadas entrantes
* sNombreSegmento = Segmentacion de llamadas entrantes
* sAtributoSegmento = Segmentacion de llamadas entrantes
* sFiltroSegmento = Segmentacion de llamadas entrantes
* idFinal = Identificador final de llamada (transaccion)
* sDescripcionFinal = Descripcion del final de llamada (transaccion)
* sFiltroFinal = Identificador + Descripcion del final de llamada (transaccion)
* tActualizado = Infor de copias de registros (info de sistemas)
* tCreacion = Fechahora inicio transaccion (entrada en sistema, no en agente) 
* nOrigenTransaccion = Entrantes siempre 2

### Importacion de los datos de envios por oficinas

In [4]:
# df4800 = pd.read_csv('data/mrw4800.txt', sep='\t', low_memory=False, encoding='latin-1')

In [5]:
# df4802 = pd.read_csv('data/mrw4802.txt', sep='\t', low_memory=False, encoding='latin-1')

In [6]:
# df4803 = pd.read_csv('data/mrw4803.txt', sep='\t', low_memory=False, encoding='latin-1')

In [7]:
# df4806 = pd.read_csv('data/mrw4806.txt', sep='\t', low_memory=False, encoding='latin-1')

In [8]:
# df4810 = pd.read_csv('data/mrw4810.txt', sep='\t', low_memory=False, encoding='latin-1')

In [9]:
# df4800.columns = df4802.columns

In [10]:
# df_shipping = pd.DataFrame()
# df_shipping = pd.concat([df_shipping, df4800])
# df_shipping = pd.concat([df_shipping, df4802])
# df_shipping = pd.concat([df_shipping, df4803])
# df_shipping = pd.concat([df_shipping, df4806])
# df_shipping = pd.concat([df_shipping, df4810])

In [11]:
# df_shipping.to_csv('data/shippings.csv')

In [12]:
df_shippings = pd.read_csv('data/shippings.csv', low_memory=False)

Los DataFrames cargados de los envios de las oficinas tienen 21 columnas:
* Fecha envío = Fecha creacion de envío
* Número envío = Identificador del envio
* Id. Fiscal = Identificacion fiscal cliente
* Nombre Comercial = Nombre del cliente
* Código servicio = Tipo de servicio, este esta vinculado con las horas en las que tendra que ser repartido
* Nombre Rem = Nombre remitente a la atencio de
* Población Rem = Poblacion remitente
* CP Rem = CP remitente
* Nombre vía Rem = Nombre domicilio remitente
* Nombre = Nombre del cliente destino
* Población = Poblacion destino
* Código postal = Codigo postal destino
* Nombre vía = Nombre domicilio cliente destino
* Total bultos = Numero de bultos en el envio
* Franquicia origen = Desde que oficina se envia el/los paquestes
* Franquicia destino = A que oficina se envia el/los paquestes
* Total = Importe envio
* Estado = Situacion envio
* Tipo anomalía = Incidencias en el envio
* Motivo = Motivo del estado del envio
* Importe Total = Importe total con gastos extras

In [13]:
import pandas as pd 
import operator
import re

In [14]:
def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    txt = re.sub(' +',' ', txt)
    return txt.strip().split()

In [15]:
def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

In [16]:
def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

In [17]:
#path = 'https://www.dropbox.com/s/edm5383iffurv4x/nombres.csv?dl=1'
#gender_list.to_csv("gender_list.csv")
gender_list = pd.read_csv("data/gender_list.csv")
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')
print(get_gender2('Margalida'))
#print(get_gender2('lopez jose maria'))
#print(get_gender2('santos contreras'))
#print(get_gender2('maria isabel lopez garcia rodriguez'))
#print(get_gender2('cami lopez'))
#print(get_gender2('majo garcia'))
#print(get_gender2('colon cristobal'))
#print(get_gender2('Cristóbal Colón'))
#print(get_gender2('Cristoforo Colombo')) 

f


In [18]:
df_calls.head(2)

,Unnamed: 0,idTransaccion,tInicio,tFinal,estado,tProximoContacto,tReplanificacion,nTAdmin,nTAdminExcedido,nTQ,idSujeto,sIdOriginal,sNombreCliente,sApellido1Cliente,sApellido2Cliente,sTelefonoCliente,idServicio,idCampanya,idAgente,idSegmento,sNombreSegmento,sAtributoSegmento,sFiltroSegmento,idFinal,sDescripcionFinal,sFiltroFinal,tActualizado,tCreacion,nOrigenTransaccion
0,0,105017008.0,NaN,2020-02-02 00:00:00.353,1.0,NaN,2020-02-03 00:00:00.000,0,0,0,0.0,0,ANONIMO,ANONIMO,NaN,NaN,0.0,100000015.0,0.0,0.0,NaN,NaN,[] [0],14.0,Finalizada por el sistema,Finalizada por el sistema [14],2020-02-02 00:01:15.507,2020-01-31 09:57:53.530,2
1,1,105017054.0,2020-01-31 10:04:20.350,2020-02-02 00:00:00.523,1.0,NaN,2020-02-03 00:00:00.000,0,0,0,0.0,0,ANONIMO,ANONIMO,NaN,NaN,100000056.0,100000021.0,100000027.0,100000030.0,Defecto,NaN,Defecto [] [100000030],14.0,Finalizada por el sistema,Finalizada por el sistema [14],2020-02-02 00:01:16.427,2020-01-31 10:04:02.723,2


In [19]:
df_callsfiltered = df_calls[['idCampanya','tCreacion']]

In [20]:
df_callsfiltered.head(3)

,idCampanya,tCreacion
0,100000015.0,2020-01-31 09:57:53.530
1,100000021.0,2020-01-31 10:04:02.723
2,100000023.0,2020-01-31 10:32:03.093


In [21]:
df_callsfiltered['Creacion'] = pd.to_datetime(df_callsfiltered['tCreacion'])

<ipython-input-21-d5df2341b303>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callsfiltered['Creacion'] = pd.to_datetime(df_callsfiltered['tCreacion'])


In [22]:
df_callsfiltered.drop('tCreacion', axis=1, inplace=True)

/home/tales/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
df_callsfiltered['Date'] = df_callsfiltered['Creacion'].dt.date

<ipython-input-23-d0f22a572b3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callsfiltered['Date'] = df_callsfiltered['Creacion'].dt.date


In [24]:
df_callsfiltered['Time'] = pd.to_datetime(df_callsfiltered['Creacion'], format='%H:%M').dt.hour

<ipython-input-24-2b158ce6b46d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callsfiltered['Time'] = pd.to_datetime(df_callsfiltered['Creacion'], format='%H:%M').dt.hour


In [25]:
df_callsfiltered

,idCampanya,Creacion,Date,Time
0,100000015.0,2020-01-31 09:57:53.530,2020-01-31,9
1,100000021.0,2020-01-31 10:04:02.723,2020-01-31,10
2,100000023.0,2020-01-31 10:32:03.093,2020-01-31,10
3,100000015.0,2020-01-31 10:52:25.230,2020-01-31,10
4,100000023.0,2020-02-02 12:55:47.980,2020-02-02,12
...,...,...,...,...
186203,100000022.0,2021-02-01 20:36:49.863,2021-02-01,20
186204,100000015.0,2021-02-01 20:37:27.950,2021-02-01,20
186205,100000021.0,2021-02-01 21:14:59.920,2021-02-01,21
186206,100000021.0,2021-02-01 21:15:43.520,2021-02-01,21


In [26]:
df_callsfiltered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186208 entries, 0 to 186207
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   idCampanya  186208 non-null  float64       
 1   Creacion    186208 non-null  datetime64[ns]
 2   Date        186208 non-null  object        
 3   Time        186208 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 5.7+ MB


In [27]:
df_callsfiltered

,idCampanya,Creacion,Date,Time
0,100000015.0,2020-01-31 09:57:53.530,2020-01-31,9
1,100000021.0,2020-01-31 10:04:02.723,2020-01-31,10
2,100000023.0,2020-01-31 10:32:03.093,2020-01-31,10
3,100000015.0,2020-01-31 10:52:25.230,2020-01-31,10
4,100000023.0,2020-02-02 12:55:47.980,2020-02-02,12
...,...,...,...,...
186203,100000022.0,2021-02-01 20:36:49.863,2021-02-01,20
186204,100000015.0,2021-02-01 20:37:27.950,2021-02-01,20
186205,100000021.0,2021-02-01 21:14:59.920,2021-02-01,21
186206,100000021.0,2021-02-01 21:15:43.520,2021-02-01,21


In [28]:
cut_labels = ['<8','8-11','11-13','13-15','15-17','17-19','19-21','>21']
cut_bins = [0,8,10,12,14,16,18,20,23]
df_callsfiltered['Interval'] = pd.cut(df_callsfiltered['Time'], bins=cut_bins, labels=cut_labels)

<ipython-input-28-f7dc496d0ca8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callsfiltered['Interval'] = pd.cut(df_callsfiltered['Time'], bins=cut_bins, labels=cut_labels)


In [29]:
df_callsfiltered

,idCampanya,Creacion,Date,Time,Interval
0,100000015.0,2020-01-31 09:57:53.530,2020-01-31,9,8-11
1,100000021.0,2020-01-31 10:04:02.723,2020-01-31,10,8-11
2,100000023.0,2020-01-31 10:32:03.093,2020-01-31,10,8-11
3,100000015.0,2020-01-31 10:52:25.230,2020-01-31,10,8-11
4,100000023.0,2020-02-02 12:55:47.980,2020-02-02,12,11-13
...,...,...,...,...,...
186203,100000022.0,2021-02-01 20:36:49.863,2021-02-01,20,19-21
186204,100000015.0,2021-02-01 20:37:27.950,2021-02-01,20,19-21
186205,100000021.0,2021-02-01 21:14:59.920,2021-02-01,21,>21
186206,100000021.0,2021-02-01 21:15:43.520,2021-02-01,21,>21


In [30]:
df_callsfiltered.groupby(['idCampanya','Date','Interval']).sum()

Time
idCampanya  Date       Interval      
100000015.0 2020-01-31 <8           0
                       8-11        19
                       11-13        0
                       13-15        0
                       15-17        0
...                               ...
100000024.0 2021-02-01 13-15      106
                       15-17       63
                       17-19       51
                       19-21        0
                       >21          0

[14600 rows x 1 columns]